In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from generateGif import create_animated_gif
from geom import find_point_C
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
class SimulationParams:
    DA_x: float
    DA_y: float
    AB_length: float
    DC_length: float
    BC_length: float
    CE_length: float
    BCE_inner_angle: float
    alpha_angle: float

In [4]:
def sim(params: SimulationParams):
    """
    Делает симуляцию
    Параметры:
    offset (tuple или list): Вектор смещения центра вращения (x0, y0).
    length (float): Длина отрезка.

    Возвращает:
    X (numpy.ndarray): Матрица координат X размером (360, 6).
    Y (numpy.ndarray): Матрица координат Y размером (360, 6).
    Порядок точек по столбцам: A, B, C, D, E, F
    """
    A_x = params.DA_x
    A_y = params.DA_y
    AB_length = params.AB_length
    углы = np.deg2rad(np.arange(0, 360))  # Углы в радианах от 0 до 359 градусов

    x_A = np.full(360, A_x)
    y_A = np.full(360, A_y)

    x_B = A_x + AB_length * np.cos(углы)
    y_B = A_y + AB_length * np.sin(углы)

    x_C = np.zeros((360))
    y_C = np.zeros((360))

    x_D = np.full(360, 0)
    y_D = np.full(360, 0)

    C_init = find_point_C(params.DC_length, (x_B[0], y_B[0]), params.BC_length)
    x_C[0] = C_init[0]
    y_C[0] = C_init[1]

    for i in range(1, 360):
        prevC = (x_C[i - 1], y_C[i - 1])
        C_this = find_point_C(
            params.DC_length, (x_B[i], y_B[i]), params.BC_length, prevIteration=prevC
        )
        x_C[i] = C_this[0]
        y_C[i] = C_this[1]

    x_E = np.zeros((360))
    x_E = np.zeros((360))

    x_BC_e = x_C - x_B
    y_BC_e = y_C - y_B
    x_BC_e /= params.BC_length
    y_BC_e /= params.BC_length

    x_CE_e = x_BC_e * math.cos(params.BCE_inner_angle) + y_BC_e * math.sin(
        params.BCE_inner_angle
    )

    y_CE_e = -x_BC_e * math.sin(params.BCE_inner_angle) + y_BC_e * math.cos(
        params.BCE_inner_angle
    )

    x_CE = x_CE_e * params.CE_length
    y_CE = y_CE_e * params.CE_length

    x_E = x_C + x_CE
    y_E = y_C + y_CE

    x_F = np.zeros((360))
    y_F = x_E * math.sin(-params.alpha_angle) + y_E * math.cos(-params.alpha_angle)

    # Создаем матрицы с повторяющимися координатами центра и вычисленными концами
    X = np.column_stack((x_A, x_B, x_C, x_D, x_E, x_F))
    Y = np.column_stack((y_A, y_B, y_C, y_D, y_E, y_F))

    return X, Y

In [41]:
params = SimulationParams()
params.AB_length = 1.1
params.DC_length = 2.2
params.BC_length = 3.2
params.DA_x = 0
params.DA_y = 3
params.BCE_inner_angle = 0.5  # радиан
params.CE_length = 1
params.alpha_angle = 0.2  # радиан

X, Y = sim(params)
create_animated_gif(X, Y, [0, 1, 2, 2, 4], [1, 2, 3, 4, 5], "tests.gif", 3000)

Анимированный GIF сохранен как tests.gif.
